In [0]:
pip install pomegranate

time: 3.58 s


In [0]:
pip install ipython-autotime

time: 3.84 s


In [0]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.22 ms


In [0]:
import time
import numpy as np
import pandas as pd
from pomegranate import *
import matplotlib.pyplot as plt
import random

time: 2.23 ms


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
time: 15.1 s


In [0]:
dataset = pd.read_csv("/content/drive/My Drive/DAMIN/HeartDisease.csv")
dataset

,Age,Sex,CP,Trestbps,Chol,Fbs,Restecg,Thalach,Exang,Oldpeak,Slope,Ca,Thal,Num (predicted attribute)
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45,1,1,110,264,0,0,132,0,1.2,2,0,7,1
299,68,1,4,144,193,1,0,141,0,3.4,2,2,7,2
300,57,1,4,130,131,0,0,115,1,1.2,2,1,7,3
301,57,0,2,130,236,0,2,174,0,0.0,2,1,3,1


time: 448 ms


In [0]:
#Hanya gunakan non categorical values
df = dataset[['Age', 'Trestbps', 'Chol', 'Thalach', 'Oldpeak']]
df

,Age,Trestbps,Chol,Thalach,Oldpeak
0,63,145,233,150,2.3
1,67,160,286,108,1.5
2,67,120,229,129,2.6
3,37,130,250,187,3.5
4,41,130,204,172,1.4
...,...,...,...,...,...
298,45,110,264,132,1.2
299,68,144,193,141,3.4
300,57,130,131,115,1.2
301,57,130,236,174,0.0


time: 19.2 ms


In [0]:
X = df.values
df

,Age,Trestbps,Chol,Thalach,Oldpeak
0,63,145,233,150,2.3
1,67,160,286,108,1.5
2,67,120,229,129,2.6
3,37,130,250,187,3.5
4,41,130,204,172,1.4
...,...,...,...,...,...
298,45,110,264,132,1.2
299,68,144,193,141,3.4
300,57,130,131,115,1.2
301,57,130,236,174,0.0


time: 17.8 ms


In [0]:
df.isna().sum()

Age         0
Trestbps    0
Chol        0
Thalach     0
Oldpeak     0
dtype: int64

time: 4.92 ms


In [0]:
if (df.isna().values.any()):
  df.dropna()

time: 1.9 ms


In [0]:
X.shape

(303, 5)

time: 2.92 ms


#KNN

In [0]:
#membuat NaN values ke dataset
random.seed(0)
np.random.seed(0)
np.set_printoptions(suppress=True)

idxs = numpy.random.choice(303, replace=False, size=5)
j =  np.random.choice(range(2,5),5)
X_nan = X.copy()
print(idxs)
print(j)

[225 152 228 201  52]
[3 4 3 3 3]
time: 4.38 ms


In [0]:
X_nan[idxs, j] = np.nan

time: 962 µs


In [0]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
df_filled = imputer.fit_transform(X_nan)
df_filled

array([[ 63. , 145. , 233. , 150. ,   2.3],
       [ 67. , 160. , 286. , 108. ,   1.5],
       [ 67. , 120. , 229. , 129. ,   2.6],
       ...,
       [ 57. , 130. , 131. , 115. ,   1.2],
       [ 57. , 130. , 236. , 174. ,   0. ],
       [ 38. , 138. , 175. , 173. ,   0. ]])

time: 468 ms


In [0]:
#check error
import sklearn.metrics as metrics

mae = metrics.mean_absolute_error(X, df_filled)
mse = metrics.mean_squared_error(X, df_filled)
rmse = np.sqrt(mse) # or mse**(0.5)

print("Results of sklearn.metrics:")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)

Results of sklearn.metrics:
MAE: 0.05500990099009899
MSE: 1.7407918151815174
RMSE: 1.3193906984595265
time: 5.79 ms


#Bayesian

In [0]:
#membuat NaN values ke dataset
random.seed(0)
np.random.seed(0)
np.set_printoptions(suppress=True)

idxs = numpy.random.choice(303, replace=False, size=5)
j =  np.random.choice(range(2,5),5)
X_nan = X.copy()
print(idxs)
print(j)

X_nan[idxs, j] = numpy.nan
# Xtemp = X_nan[:500]

[225 152 228 201  52]
[3 4 3 3 3]
time: 6.38 ms


In [0]:
X_nan.shape

(303, 5)

time: 2.29 ms


In [0]:
model1 = BayesianNetwork.from_samples(X_nan, max_parents=5)

time: 6.39 s


In [0]:
model1.predict_proba({})

array([{
    "class" :"Distribution",
    "dtype" :"numpy.float64",
    "name" :"DiscreteDistribution",
    "parameters" :[
        {
            "63.0" :0.029702970297029667,
            "67.0" :0.029702970297029667,
            "37.0" :0.006600660066006766,
            "41.0" :0.03300330033003294,
            "56.0" :0.03630363036303621,
            "62.0" :0.03630363036303621,
            "57.0" :0.05610561056105584,
            "53.0" :0.026402640264026396,
            "44.0" :0.03630363036303621,
            "52.0" :0.04290429042904275,
            "48.0" :0.02310231023102312,
            "54.0" :0.05280528052805257,
            "49.0" :0.01650165016501658,
            "64.0" :0.03300330033003294,
            "58.0" :0.06270627062706238,
            "60.0" :0.039603960396039486,
            "50.0" :0.02310231023102312,
            "66.0" :0.02310231023102312,
            "43.0" :0.026402640264026396,
            "40.0" :0.009900990099010036,
            "69.0" :0.00990099009901003

time: 12.6 ms


In [0]:
y_pred1 = model1.predict(X_nan)

time: 1.3 s


In [0]:
#check error
import sklearn.metrics as metrics

mae = metrics.mean_absolute_error(X, np.array(y_pred1))
mse = metrics.mean_squared_error(X, np.array(y_pred1))
rmse = np.sqrt(mse) # or mse**(0.5)

print("Results of sklearn.metrics:")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)

Results of sklearn.metrics:
MAE: 0.06772277227722771
MSE: 2.6162112211221125
RMSE: 1.6174706245005233
time: 7.25 ms


#Deterministic Regression

In [0]:
df2 = dataset[['Age', 'Trestbps', 'Chol', 'Thalach', 'Oldpeak']]
X2 = df2.values
df2

,Age,Trestbps,Chol,Thalach,Oldpeak
0,63,145,233,150,2.3
1,67,160,286,108,1.5
2,67,120,229,129,2.6
3,37,130,250,187,3.5
4,41,130,204,172,1.4
...,...,...,...,...,...
298,45,110,264,132,1.2
299,68,144,193,141,3.4
300,57,130,131,115,1.2
301,57,130,236,174,0.0


time: 21.6 ms


In [0]:
#membuat NaN values ke dataset
random.seed(0)
np.random.seed(0)
np.set_printoptions(suppress=True)

idxs = np.random.choice(303, replace=False, size=5)
j =  np.random.choice(range(2,5),5)
X_nan = X2.copy()
print(idxs)
print(j)
X_nan[idxs, j] = numpy.nan
df_nan = pd.DataFrame(X_nan, columns=['Age', 'Trestbps', 'Chol', 'Thalach', 'Oldpeak'])
df_nan

[225 152 228 201  52]
[3 4 3 3 3]


,Age,Trestbps,Chol,Thalach,Oldpeak
0,63.0,145.0,233.0,150.0,2.3
1,67.0,160.0,286.0,108.0,1.5
2,67.0,120.0,229.0,129.0,2.6
3,37.0,130.0,250.0,187.0,3.5
4,41.0,130.0,204.0,172.0,1.4
...,...,...,...,...,...
298,45.0,110.0,264.0,132.0,1.2
299,68.0,144.0,193.0,141.0,3.4
300,57.0,130.0,131.0,115.0,1.2
301,57.0,130.0,236.0,174.0,0.0


time: 26.9 ms


In [0]:
df_nan.isna().sum()

Age         0
Trestbps    0
Chol        0
Thalach     4
Oldpeak     1
dtype: int64

time: 5.74 ms


In [0]:
missing_columns = ['Thalach', 'Oldpeak']
deter_data = pd.DataFrame(columns = ["Det" + name for name in missing_columns])
from sklearn.linear_model import LinearRegression
for feature in missing_columns:
  # print(feature)
  deter_data["Det" + feature] = df_nan[feature]
  parameters = list(set(df_nan.columns) - set(missing_columns) - {feature})

  # #Create a Linear Regression model to estimate the missing data
  model = LinearRegression()
  model.fit(X = df2[parameters], y = df2[feature])

  #observe that I preserve the index of the missing data from the original dataframe
  deter_data.loc[df_nan[feature].isna(), "Det" + feature] = model.predict(df_nan[parameters])[df_nan[feature].isna()]

time: 53.9 ms


In [0]:
deter_data

,DetThalach,DetOldpeak
0,150.0,2.3
1,108.0,1.5
2,129.0,2.6
3,187.0,3.5
4,172.0,1.4
...,...,...
298,132.0,1.2
299,141.0,3.4
300,115.0,1.2
301,174.0,0.0


time: 16.1 ms


In [0]:
non_missing_columns = ['Age', 'Trestbps', 'Chol', 'Thalach', 'Oldpeak']
data_non_nan = pd.DataFrame(columns = ["Det" + name for name in non_missing_columns])

for feature in non_missing_columns:
  data_non_nan["Det" + feature] = df2[feature]

for feature in missing_columns:
  data_non_nan["Det" + feature] = deter_data["Det" + feature]
data_non_nan

,DetAge,DetTrestbps,DetChol,DetThalach,DetOldpeak
0,63,145,233,150.0,2.3
1,67,160,286,108.0,1.5
2,67,120,229,129.0,2.6
3,37,130,250,187.0,3.5
4,41,130,204,172.0,1.4
...,...,...,...,...,...
298,45,110,264,132.0,1.2
299,68,144,193,141.0,3.4
300,57,130,131,115.0,1.2
301,57,130,236,174.0,0.0


time: 34.1 ms


In [0]:
#check error
import sklearn.metrics as metrics

mae = metrics.mean_absolute_error(X, np.array(data_non_nan))
mse = metrics.mean_squared_error(X, np.array(data_non_nan))
rmse = np.sqrt(mse) # or mse**(0.5)

print("Results of sklearn.metrics:")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)

Results of sklearn.metrics:
MAE: 0.05119646234924711
MSE: 1.413132533551361
RMSE: 1.1887525114805693
time: 6.39 ms


#Summary

##Heart Disease

###KNN


####Results of sklearn.metrics:
####MAE: 0.05500990099009899
####MSE: 1.7407918151815174
####RMSE: 1.3193906984595265

###Bayesian Network

####Results of sklearn.metrics:
####MAE: 0.06772277227722771
####MSE: 2.6162112211221125
####RMSE: 1.6174706245005233

###Deterministic Regression

####Results of sklearn.metrics:
####MAE: 0.05119646234924711
####MSE: 1.413132533551361
####RMSE: 1.1887525114805693